# **ChEMBL Data Import and Preparation**

## Objectives:

The first notebook focuses on setting up the data pipeline by:

1.  Reading the ChEMBL dataset directly into our Azure MySQL Database from the ChEMBL URL.

2.  Taking the new raw data, querying it for what we need at this time, and placing it in a separate database.

3.  Examines this new set of data for duplicates and missing values, removing as needed.

4.  Saving our work so that we can use this version of the data for the rest of this phase of the project, saving also as a parquet file.



### Section 1: Import Libraries and Establish Project Root for Directory

##### First let's set our directory to the root of the project.  Doing so will keep our project on track location-wise and it is a great way to keep yourself out of trouble with your directory issues, by setting your abspath to the root.  We can then in future notebooks refer back to the project_root when needed.

In [1]:
import os

# Define project root
project_root = "/home/azureuser/cloudfiles/code/Users/kalpha1865/BioPred"

# Validate the directory
if not os.path.exists(project_root):
    raise FileNotFoundError(f"Project root not found: {project_root}")

# Change working directory to project root if not already
if os.getcwd() != project_root:
    os.chdir(project_root)

print(f"Project root set to: {os.getcwd()}")

Project root set to: /mnt/batch/tasks/shared/LS_root/mounts/clusters/kalpha18651/code/Users/kalpha1865/BioPred


##### Now we can import the rest of our libraries, as well as establish a reference point to our Config file for our database credentials.

In [2]:
import sys
import requests
import subprocess
import pandas as pd
import tarfile
from azure.storage.blob import BlobServiceClient
import fastparquet
from sqlalchemy import create_engine, text, Index, MetaData, Table, inspect
import mysql.connector


# Referencing the config file for Azure MySQL Database credentials.
config_dir = os.path.join(project_root, "Config")
sys.path.append(config_dir)
print(f"Config directory: {config_dir}")
print("Files in Config directory:", os.listdir(config_dir))

from config import MYSQL_CONFIG


Config directory: /home/azureuser/cloudfiles/code/Users/kalpha1865/BioPred/Config
Files in Config directory: ['.amlignore', '.amlignore.amltmp', 'config.json', 'config.py', '__pycache__']


### Section 2: Read and Extract Data from URL

##### Now we will bring in our ChEMBL data, sending it straight from the site url to our Azure MySQL Database.  We will read in our credentials from a config file for access.  The commands used in the function below can normally be used in the terminal however I wanted to show my work here.  First though we will need to create our databases that we will use in this portion of the project to house and work with the data.

In [3]:
# setting up as a try/except block so we can add error handling.
try:
    # Set up the connection string to Azure
    engine = create_engine(
        f"mysql+mysqlconnector://{MYSQL_CONFIG['username']}:{MYSQL_CONFIG['password']}@"
        f"{MYSQL_CONFIG['hostname']}:{MYSQL_CONFIG['port']}/",
        connect_args={
            "ssl_ca" : MYSQL_CONFIG["ssl_ca"],
            "ssl_verify_cert" : True
        }
    )

    # Create the databases needed
    with engine.connect() as connection:
        connection.execute(text("CREATE DATABASE IF NOT EXISTS chembl_raw;"))
        print("Database 'chembl_raw' created successfully.")
    
        connection.execute(text("CREATE DATABASE IF NOT EXISTS chembl_phase_1;"))
        print("Database 'chembl_phase_1' created successfully.")
    
        # Verify databases exist.
        result = connection.execute(text("SHOW DATABASES;"))
        databases = [row[0] for row in result]
        
        if "chembl_raw" in databases and "chembl_phase_1" in databases:
            print("Verification successful: Databases exist in the Azure MySQL server.")
        else:
            print("Error: Databases were not found after creation.")
except Exception as e:
    print(f"An error occurred: {e}")

Database 'chembl_raw' created successfully.
Database 'chembl_phase_1' created successfully.
Verification successful: Databases exist in the Azure MySQL server.


##### Great the new databases are created and housed in our Azure MySQL server (I verified on the Azure portal as well).  Now let's fetch the url containing our data from ChEMBL and send it to our empty database, so we can query off of it and get what we need for phase_1 data. 

In [4]:
# Function to fetch, extract, and send the contents to our db.

def prepare_and_load_data():
    tar_file = "/home/azureuser/cloudfiles/code/Users/kalpha1865/BioPred/chembl_35_mysql.tar.gz"
    extract_dir = "/home/azureuser/cloudfiles/code/Users/kalpha1865/BioPred/chembl_35/chembl_35_mysql/"
    dmp_file = os.path.join(extract_dir, "chembl_35_mysql.dmp")

    # Verify tar file
    if not os.path.exists(tar_file):
        print(f"{tar_file} not found. Downloading...")
        subprocess.run(
            f"wget ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_35_mysql.tar.gz -O {tar_file}",
            shell=True
        )
    
    # Clean up existing files
    if os.path.exists(extract_dir):
        print(f"Removing existing directory: {extract_dir}")
        subprocess.run(f"rm -rf {extract_dir}", shell=True)
    
    # Extract tar file
    print("Extracting tar file...")
    process = subprocess.run(f"tar -xzf {tar_file} -C /home/azureuser/cloudfiles/code/Users/kalpha1865/BioPred/", shell=True)
    if process.returncode != 0:
        print("Error during extraction.")
        return
    
    # Verify .dmp file
    if not os.path.exists(dmp_file):
        print(f"{dmp_file} not found after extraction.")
        return
    
    print("Loading .dmp file into MySQL...")
    load_command = (
        f"mysql -h biopred.mysql.database.azure.com -u rdm1 -p'tali1327_yo' "
        f"--ssl-mode=VERIFY_CA --ssl-ca=/home/azureuser/cloudfiles/code/Users/kalpha1865/BioPred/docs/certs/DigiCertGlobalRootCA.crt.pem "
        f"-D chembl_raw < {dmp_file}"
    )
    process = subprocess.run(load_command, shell=True)
    if process.returncode == 0:
        print("Data successfully loaded into MySQL.")
    else:
        print(f"Error loading data. Return code: {process.returncode}")

prepare_and_load_data()



Removing existing directory: /home/azureuser/cloudfiles/code/Users/kalpha1865/BioPred/chembl_35/chembl_35_mysql/
Extracting tar file...


rm: cannot remove '/home/azureuser/cloudfiles/code/Users/kalpha1865/BioPred/chembl_35/chembl_35_mysql/': Directory not empty


Error during extraction.


tar: chembl_35/chembl_35_mysql/chembl_35_mysql.dmp: Cannot open: No such file or directory
tar: Exiting with failure status due to previous errors


##### Now that we have the data sent to our database let's create a new connection and check the table names.

In [5]:

# Define a new engine for the connection, specifying our new database for the raw data.
raw_engine = create_engine(
        f"mysql+mysqlconnector://{MYSQL_CONFIG['username']}:{MYSQL_CONFIG['password']}@"
        f"{MYSQL_CONFIG['hostname']}:{MYSQL_CONFIG['port']}/{MYSQL_CONFIG['database_raw']}",
        connect_args={
            "ssl_ca" : MYSQL_CONFIG["ssl_ca"],
            "ssl_verify_cert" : True
        }
    )

with raw_engine.connect() as connection:
    result = connection.execute(text("SHOW TABLES;"))
    print("Tables in raw database:", [row[0] for row in result.fetchall()])

Tables in raw database: ['action_type', 'activities', 'activity_properties', 'activity_smid', 'activity_stds_lookup', 'activity_supp', 'activity_supp_map', 'assay_class_map', 'assay_classification', 'assay_parameters', 'assay_type', 'assays', 'atc_classification', 'binding_sites', 'bio_component_sequences', 'bioassay_ontology', 'biotherapeutic_components', 'biotherapeutics', 'cell_dictionary', 'chembl_id_lookup', 'chembl_release', 'component_class', 'component_domains', 'component_go', 'component_sequences', 'component_synonyms', 'compound_properties', 'compound_records', 'compound_structural_alerts', 'compound_structures', 'confidence_score_lookup', 'curation_lookup', 'data_validity_lookup', 'defined_daily_dose', 'docs', 'domains', 'drug_indication', 'drug_mechanism', 'drug_warning', 'formulations', 'frac_classification', 'go_classification', 'hrac_classification', 'indication_refs', 'irac_classification', 'ligand_eff', 'mechanism_refs', 'metabolism', 'metabolism_refs', 'molecule_atc_

### Section 3: Querying Our New Data

##### Let's take a look at the schema to get a visual for the ChEMBL data. Seen below.  We will use this to formulate our query and our indexes for the next part.

<img src="https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_35_schema.png" alt = "ChEMBL Schema" width = 2000>

##### Next we will set a few indexes for our data; this will help considerably when we go to query our data.  We will also set indexes with a general theme, setting indexes for features we will need and use throughout this project so we don't change them again as this is the raw data we will be iterating on.

##### Before creating our indexes though we need to map the MetaData of our tables that we will need for our joins in our query.  This will allow us to query a lot faster through our forthcoming indexes, as with those we won't need to scan the whole dataset every time.

In [6]:
metadata = MetaData()

metadata.reflect(bind=raw_engine)

# Reflect the tables to the metadata.
compound_structures = Table("compound_structures", metadata, autoload_with=raw_engine)
activities = Table("activities", metadata, autoload_with=raw_engine)
assays = Table("assays", metadata, autoload_with=raw_engine)
target_dictionary = Table("target_dictionary", metadata, autoload_with=raw_engine)
compound_properties = Table("compound_properties", metadata, autoload_with=raw_engine)


InternalError: (mysql.connector.errors.InternalError) 1213 (40001): Deadlock found when trying to get lock; try restarting transaction
[SQL: SHOW CREATE TABLE `assays`]
(Background on this error at: https://sqlalche.me/e/20/2j85)

##### And a check to see existing indexes.

In [ ]:
inspector = inspect(raw_engine)
indexes = inspector.get_indexes("compound_structures") # Example table
print(indexes)


##### Now let's create a function to add our new indexes.

In [ ]:
# Instantiate function to create new index if it doesn't exist
def create_index(table_name, index_name, column_name):
    inspector = inspect(raw_engine)
    existing_indexes = [idx["name"] for idx in inspector.get_indexes(table_name)]
    if index_name not in existing_indexes:
        Index(index_name, column_name).create(raw_engine)
        print(f"Index {index_name} created successfully.")
    else:
        print(f"Index {index_name} already exists. Skipping creation.")

# Use function to add new wanted indexes

# First the indexes for the joins
create_index("compound_structures", "ix_molregno", compound_structures.c.molregno)
create_index("activities", "ix_activities_molregno", activities.c.molregno)
create_index("assays", "ix_assay_id", assays.c.assay_id)
create_index("target_dictionary", "ix_tid", target_dictionary.c.tid)

# Now the indexes for filtering
create_index("compound_properties", "ix_full_mwt", compound_properties.c.full_mwt)
create_index("compound_properties", "ix_hba_lipinski", compound_properties.c.hba_lipinski)
create_index("compound_properties", "ix_hbd_lipinski", compound_properties.c.hbd_lipinski)
create_index("compound_properties", "ix_alogp", compound_properties.c.alogp)
create_index("compound_properties", "ix_psa", compound_properties.c.psa)
create_index("compound_properties", "ix_rtb", compound_properties.c.rtb)



##### We will first attempt to run the whole query as it is first, then slowly editing and changing as we see results based on duplicate and missing value numbers.  The query below is our final product after numerous iterations, as we relaxed our parameters from Lipinski's Rule of Five set to allow for an increase in data allotment and made sure that our pref_name feature was intact without pulling in a lot of missing values.

In [ ]:

# Query the raw data to get what we need in our phase_1_db.
def query_phase_1_data():
    # Targeted query for phase 1 data with subqueries for better processing
    query = text(f"""           
    WITH phase_1_cte AS(
        SELECT DISTINCT cs.canonical_smiles, cs.molregno,
            MIN(cp.full_mwt) AS full_mwt, MIN(cp.alogp) AS alogp,
            MIN(cp.psa) AS psa, MIN(cp.hba_lipinski) AS hba_lipinski, MIN(cp.hbd_lipinski) AS hbd_lipinski,
            MIN(cp.aromatic_rings) AS aromatic_rings, MIN(cp.heavy_atoms) AS heavy_atoms,
            MIN(cp.rtb) AS rtb, cp.molecular_species, fa.min_standard_value,
            td.pref_name, td.tid, td.target_type
        FROM compound_structures cs
        LEFT JOIN compound_properties cp ON cs.molregno = cp.molregno
        LEFT JOIN (
            SELECT molregno, MIN(standard_value) AS min_standard_value, assay_id
            FROM activities
            WHERE standard_value BETWEEN 0 AND 500
                AND standard_type IN ('IC50', 'EC50')
            GROUP BY molregno, assay_id
        ) fa ON cs.molregno = fa.molregno
        LEFT JOIN assays ass ON fa.assay_id = ass.assay_id
        LEFT JOIN target_dictionary td ON ass.tid = td.tid
        WHERE
            cp.full_mwt BETWEEN 100 AND 600
            AND cp.alogp BETWEEN -1 AND 6
            AND cp.psa <= 180
            AND cp.rtb <= 15
            AND cp.hbd_lipinski <= 7
            AND cp.hba_lipinski <= 15
            AND td.pref_name IS NOT NULL
        GROUP BY
            cs.canonical_smiles,
            cp.molecular_species,
            td.pref_name,
            td.tid,
            td.target_type,
            fa.min_standard_value
        )
        SELECT
            canonical_smiles,
            molregno,
            full_mwt,
            alogp,
            psa,
            hba_lipinski,
            hbd_lipinski,
            aromatic_rings,
            heavy_atoms,
            rtb,
            molecular_species,
            min_standard_value,
            pref_name,
            tid,
            target_type,
            RANK() OVER (ORDER BY min_standard_value ASC) AS rank_min_standard_value,
            RANK() OVER(PARTITION BY molregno ORDER BY min_standard_value ASC) AS rank_molregno_min_standard_value,
            RANK() OVER(PARTITION BY tid ORDER BY min_standard_value ASC) AS rank_tid_min_std_value
        FROM phase_1_cte;
        """)


    # Execute query with error handling
    try:
        # Execute query and fetch results
        with raw_engine.connect() as connection:
            result = connection.execute(query) 
            df_phase_1 = pd.DataFrame(result.fetchall(), columns = result.keys())
            print(f"Query returned dataset shape: {df_phase_1.shape}")
            
            # Drop the molregno feature as we no longer need.
            df_phase_1.drop(columns = ['molregno'], inplace = True)
            
            return df_phase_1

    except Exception as e:
            print(f"Error querying phase 1 data: {e}")
            return pd.DataFrame()
    

# Check the data for duplicates and missing values before saving.
def check_data_quality(df):
    if df.empty:
        print("DataFrame is empty.  Skipping quality checks.")
        return
    
    print("Checking for duplicates...")
    print(f"Number of duplicate canonical_smiles: {df['canonical_smiles'].duplicated().sum()}")
    
    print("\nChecking for missing values...")
    print(df.isna().sum())

# Now run both functions
df_phase_1 = query_phase_1_data()
check_data_quality(df_phase_1)

##### Now we can remove the duplicates and the small amount of rows in molecular_species.  We don't want any duplicated data in canonical_smiles as that is going to be our target feature, and the amount of missing values in molecular_species is negligible (0.4%) so there isn't much reason to look into that at this time and removing them to have a clean df is optimal at this time.

In [ ]:
df_phase_1 = df_phase_1.drop_duplicates(subset = "canonical_smiles")
df_phase_1 = df_phase_1.dropna(subset = ['molecular_species'])
print(df_phase_1.shape)
print(df_phase_1.head())

##### Just around ~343k rows after the query and clean.  I am okay with this as I am opting for quality data for this phase of the project and it will help our modeling.  We also will have a lot of feature engineering and feature formatting and manipulation to do so this data will be expanding.  Let's save it and finish up.

##### First we need to make another connection engine for phase_1_data.

In [ ]:
# Define a new engine for the connection, specifying our new database for the phase_! data.
phase_1_engine = create_engine(
        f"mysql+mysqlconnector://{MYSQL_CONFIG['username']}:{MYSQL_CONFIG['password']}@"
        f"{MYSQL_CONFIG['hostname']}:{MYSQL_CONFIG['port']}/{MYSQL_CONFIG['database_phase_1']}",
        connect_args={
            "ssl_ca" : MYSQL_CONFIG["ssl_ca"],
            "ssl_verify_cert" : True
        }
    )

print("Phase 1 database engine created successfully.")

In [ ]:
# Try/except block here to save to the MySQL database
try:
    df_phase_1.to_sql(
        name="df_phase_1_data",
            con=phase_1_engine,
        if_exists="replace",
        index = False,
        chunksize= 10000        
    )
    print(f"Results successfully saved to phase_1_data")
except Exception as e:
    print(f"Error saving to target database: {e}")


##### We will also save as a parquet file so we can carry over and use in our EDA notebook next.

In [ ]:

file_path = "/home/azureuser/cloudfiles/code/Users/kalpha1865/BioPred/Data/df_files/df_phase_1.parquet"
df_phase_1.to_parquet(file_path, index = False)
print("Queried phase 1 data saved as Parquet in df_files in Data folder.")

##### As a final step we will dispose of our connections to our engine(s).

In [ ]:
raw_engine.dispose()
phase_1_engine.dispose()

print("All database connections have been closed.")